# Преобразования

Важной частью визуализации являются преобразования данных.

При использовании Altair, у вас есть два возможных пути для преобразования данных:

1. предварительные преобразования в Python
2. преобразование в Altair/Vega-Lite

In [1]:
import altair as alt

## Вычисление преобразований

В качестве примера, рассмотрим преобразования данных, представленных не в интуитивно понятной форме.
Набор данных ``population`` содержит списки по США, классифицированные по годам, полу, возрасту, но пол обозначен цифрами "1" и "2", что делает графики менее интуитивно понятными:

In [2]:
from vega_datasets import data
population = data.population()

In [3]:
population.head()

,year,age,sex,people
0,1850,0,1,1483789
1,1850,0,2,1450376
2,1850,5,1,1411067
3,1850,5,2,1359668
4,1850,10,1,1260099


In [4]:
alt.Chart(population).mark_bar().encode(
    x='year:O',
    y='sum(people):Q',
    color='sex:N'
)

alt.Chart(...)

Один из способов исправить это в Python – использовать средства Pandas для переобозначения значений в этом столбце. Например:

In [5]:
population['men_women'] = population['sex'].map({1: 'Men', 2: 'Women'})

alt.Chart(population).mark_bar().encode(
    x='year:O',
    y='sum(people):Q',
    color='men_women:N'
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

Но Altair создан, чтобы работать с данными на основе URL, а в этом случае предобработка будет невозможной.
В таких ситуациях, лучше сделать преобразование частью спецификации графика.
Это может быть с использованием метода ``transform_calculate``, который принимает [выражение Vega](https://vega.github.io/vega/docs/expressions/), представляющее собой строку с несколькими операциями javascript:

In [7]:
# undo our addition of a column above...
population = population.drop('men_women', axis=1)

In [10]:
alt.Chart(population).mark_bar().encode(
    x='year:O',
    y='sum(people):Q',
    color='men_women:N'
).transform_calculate(
    men_women='datum.sex == 1 ? "Men" : "Women"'
)

alt.Chart(...)

In [11]:
population

,year,age,sex,people
0,1850,0,1,1483789
1,1850,0,2,1450376
2,1850,5,1,1411067
3,1850,5,2,1359668
4,1850,10,1,1260099
...,...,...,...,...
565,2000,80,2,3221898
566,2000,85,1,970357
567,2000,85,2,1981156
568,2000,90,1,336303


Потенциальным непонятным местом может быть наличие слова"datum": но это просто установленное в Vega обозначением строк данных.

Если вы предпочитаете использовать выражения на Python, Altair предоставляет API и для этого:

In [12]:
from altair.expr import datum, if_

alt.Chart(population).mark_bar().encode(
    x='year:O',
    y='sum(people):Q',
    color='men_women:N'
).transform_calculate(
    men_women=if_(datum.sex == 1, "Men", "Women")
)

alt.Chart(...)

## Фильтрующее преобразование

Фильтрующее преобразование выполняется аналогично. Пусть, например, мы хотим составить график, включающий только мужское население.
Как и раньше, это можно сделать с помощью Pandas, но будет полезнее иметь такое преобразование в спецификации графика.
Это может быть сделать функцией ``transform_filter()``:

In [13]:
alt.Chart(population).mark_bar().encode(
    x='year:O',
    y='sum(people):Q',
).transform_filter(
    datum.sex == 1
)

alt.Chart(...)

Мы уже использовали метод ``transform_filter`` раньше, когда фильтровали данные на основе выделения\.

## Другие преобразования

Примеры других преобразований можно найти в документации Altair [по преобразованиям](https://altair-viz.github.io/user_guide/transform/index.html).

Можно перечислить несколько наиболее распространённых преобразований Altair:

- ``transform_aggregate()``
- ``transform_bin()``
- ``transform_timeunit()``

Эти три преобразования делают ровно те же операции, которые мы рассматривали в секции, посвящённой объединению и агрегации данных, с тем отличием, что в результате этих преобразований формируется новое именованное значение, которое можно использовать в нескольких местах графика.



Кроме того, существует множество других преобразований, например:

- ``transform_lookup()``: позволяет выполнять одностороннее объединение нескольких наборов данных.
- ``transform_window()``: позволяет выполнять опреации агрегации в скользящем окне.

## Упражение

Для следующего набора:

In [14]:
import pandas as pd
import numpy as np
x = pd.DataFrame({'x': np.linspace(-5, 5)})

1. Создать график, основанные на этих данных и сформировать график синуса и косинуса с помощью функции Altair ``transform_calculate``.
2. Использовать ``transform_filter`` на этом графике, и удалить области графика, где значение косинуса меньше значения синуса.

In [27]:
alt.Chart(x).transform_calculate(
    sin_value='sin(datum.x)',  # Расчитаем синус
    cos_value='cos(datum.x)'   # и косинус
).transform_fold(
    ['sin_value', 'cos_value'],  # Переводим столбцы в длинный формат
    as_=['function', 'value']    # Новый столбец для функции и значений
).mark_line().encode(
    x='x:Q',
    y='value:Q',                 # Ось Y - синус и косинус
    color='function:N',
)

alt.Chart(...)

Значения косинуса меньше синуса удалились, но при построении графика он соединил две точки прямой линией

In [42]:
alt.Chart(x).transform_calculate(
    sin_value='sin(datum.x)',
    cos_value='cos(datum.x)'
).transform_fold(
    ['sin_value', 'cos_value'],
    as_=['function', 'value']
).transform_filter(
    'datum.function == "sin_value" | datum.cos_value >= datum.sin_value'  # Фильтрация по условию косинус >= синус
).mark_line().encode(
    x='x:Q',
    y='value:Q',
    color='function:N',
)

alt.Chart(...)

Можно проверить это, построив график точками например ``mark_circle``

In [43]:
alt.Chart(x).transform_calculate(
    sin_value='sin(datum.x)',
    cos_value='cos(datum.x)'
).transform_fold(
    ['sin_value', 'cos_value'],
    as_=['function', 'value']
).transform_filter(
    'datum.function == "sin_value" | datum.cos_value >= datum.sin_value'  # Фильтрация по условию косинус >= синус
).mark_circle().encode(
    x='x:Q',
    y='value:Q',
    color='function:N',
)

alt.Chart(...)